<a href="https://colab.research.google.com/github/julianovale/BB_Evolution/blob/main/0040_AluraBB_ML_ValidacaoModelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando a base de dados

In [1]:
import pandas as pd

In [2]:
uri = 'https://raw.githubusercontent.com/julianovale/BB_Evolution/main/Arquivos/machine-learning-carros-simulacao.csv'
dados = pd.read_csv(uri)
dados.head()

,Unnamed: 0,preco,vendido,idade_do_modelo,km_por_ano
0,0,30941.02,1,18,35085.22134
1,1,40557.96,1,20,12622.05362
2,2,89627.50,0,12,11440.79806
3,3,95276.14,0,3,43167.32682
4,4,117384.68,1,4,12770.11290


In [3]:
dados = dados.drop(columns=['Unnamed: 0'], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


# Separando x e y

In [4]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

# Dividindo treinamento e teste

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


# Criando o *baseline*

In [6]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy='stratified')
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 50.96%


# Tentando um algoritmo

In [7]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [8]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia do dummy stratified foi 76.84%


Duas sementes aleatórias geraram duas acurácias distintas. Não basear minhas decisões em um único ensaio, então. O que posso fazer?

# Validação cruzada

Anteriormente, percebemos que, se usarmos a técnica de separar uma parte dos dados para treino e outra para testes, seguramos uma parte dos dados. Em inglês, isto é chamado de **Holdout**.

O principal aqui é a ideia de que quebrarmos os dados em N pedaços e, então, rodarmos K processos de treino e teste, validando nosso algoritmo de maneira cruzada. Esse processo é chamado de **K-fold**, referente ao número (K) de vezes que quebraremos os dados para rodar a validação cruzada.

Por fim, pense em um caso extremo, no qual temos 12 elementos e os quebramos ao máximo, em 12 pedaços. Assim, em um dos 12 treinos e testes, treinaremos com 11 deles e testaremos com 1. Ou seja, vamos testar o número de elementos que tivermos N vezes. Este algoritmo será muito lento, porque esse processo de validar, deixando um isolado, nos obriga a rodá-lo K vezes (o número de elementos que possuímos).

Entretanto, o processo é o mais fiel ao que esperamos com esse algoritmo no mundo real. Sendo assim, é muito importante decidirmos qual será o número K. Esse caso específico, em que o número K é exatamente o número de elementos que temos, é chamado de **Leave One Out** -"deixar um de fora", em português -, justamente por deixar um elemento de fora, isolado.

In [9]:
from sklearn.model_selection import cross_validate


SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3)
results

{'fit_time': array([0.01200891, 0.00788426, 0.00765276]),
 'score_time': array([0.00316739, 0.002563  , 0.00249386]),
 'test_score': array([0.75704859, 0.7629763 , 0.75337534])}

In [10]:
results['test_score']

array([0.75704859, 0.7629763 , 0.75337534])

In [11]:
media = results['test_score'].mean()
media

0.7578000751484867

In [12]:
desviopadrao = results['test_score'].std()
desviopadrao

0.003955431356145979

## Gerando um intervalo

In [13]:
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy %.2f %.2f" % (media - 2 * desvio_padrao, media + 2 * desvio_padrao))

Accuracy 0.75 0.77


In [14]:
print("Accuracy com cross validation, 3 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 3 = [74.99, 76.57]


### k = 3

In [15]:
from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 3 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 3 = [74.99, 76.57]


### k = 5

In [16]:
from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 5 = [75.21, 76.35]


### k = 10

In [17]:
from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 10 = [74.24, 77.32]


### Quantos k usar?

Então, qual número devemos usar para o cross validation (cv)? Alguns papers científicos, encontrados na documentação da versão usada no vídeo, apontam que escolher 5 ou 10 para cv já é o suficiente. Sendo assim, vamos manter o valor de cv do nosso código como 5, que gerou o intervalo de 75.21 a 76.35, para a taxa de acerto que acreditamos alcançar com o algoritmo que estamos utilizando, como demonstrado logo acima.

Vamos adotar k = 5.

In [18]:
from sklearn.model_selection import cross_validate

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results ['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 5 = [75.21, 76.35]


# 'Embaralhando' os dados para fazer o K-fold (validação cruzada)
# Aleatoriedade no cross validate

In [19]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 10 = [74.37, 77.19]


In [20]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio = [%.2f]" % (media*100))
    print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

In [21]:
from sklearn.model_selection import KFold
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio = [75.78]
Accuracy com cross validation, 10 = [74.37, 77.19]


### Shuffle (embaralhar)

In [22]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy médio = [%.2f]" % (media*100))
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy médio = [75.76]
Accuracy com cross validation, 10 = [73.26, 78.26]


# Estratificação com validação cruzada

## Importante para distribuição desbalanceada de classes

## StratifiedKFold

In [23]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy médio = [%.2f]" % (media*100))
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy médio = [75.78]
Accuracy com cross validation, 10 = [74.42, 77.14]


Ver link:

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection

# Validação cruzada com grupos

In [24]:
dados

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290
...,...,...,...,...
9995,97112.86,0,12,25060.64248
9996,107424.63,1,16,21317.31764
9997,93856.99,0,4,20950.38812
9998,51250.57,1,7,16840.13376


In [25]:
dados['idade_do_modelo'].describe()

count    10000.00000
mean        13.85580
std          4.68758
min          1.00000
25%         11.00000
50%         15.00000
75%         18.00000
max         20.00000
Name: idade_do_modelo, dtype: float64

### criando uma coluna com o modelo (aleatório) do carro

In [26]:
np.random.seed(SEED)
dados['modelo'] = np.random.randint(-2, 3, size=10000) + dados['idade_do_modelo']
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,16
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,3


In [27]:
dados['modelo'].unique()

array([16, 22, 12,  4,  3, 11, 18, 17, 13,  0, 15, 10,  9, 14,  1,  5, 19,
       21,  8,  7, 20,  6,  2, -1])

Quero excluir o -1. Como faço?

In [28]:
dados['modelo'] = dados['modelo'] + abs(dados['modelo'].min())+1

In [29]:
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [30]:
dados['modelo'].unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

In [31]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, groups=dados['modelo'], return_train_score=False) # precisa indicar a coluna com os grupos aqui (groups = coluna)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy médio = [%.2f]" % (media*100))
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy médio = [75.80]
Accuracy com cross validation, 10 = [72.00, 79.60]


# Verificando outros classificadores
# Cross Validation com Standard Scaler

In [32]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 77.48%


In [37]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
scaler = StandardScaler()
scaler.fit(x)
x_escalado = scaler.transform(x)

modelo = SVC()
results = cross_validate(modelo, x, y, cv = cv, groups=dados['modelo'], return_train_score=False) # precisa indicar a coluna com os grupos aqui (groups = coluna)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy médio = [%.2f]" % (media*100))
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy médio = [77.23]
Accuracy com cross validation, 10 = [73.75, 80.70]


## Pipeline

Sendo assim, no Sklearn, criaremos um processo, uma sequência de passos, chamada Pipeline.

O legal do Pipeline é que ele funciona como se fosse um estimador, então tem o fit() e o predict, que funciona da mesma forma. Então, podemos passar ele para o processo de cross_validate.

In [39]:
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])
pipeline

Pipeline(steps=[('transformacao', StandardScaler()), ('estimador', SVC())])

In [40]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits = 10)

results = cross_validate(pipeline, x, y, cv = cv, groups=dados['modelo'], return_train_score=False) # precisa indicar a coluna com os grupos aqui (groups = coluna)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy médio = [%.2f]" % (media*100))
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy médio = [76.55]
Accuracy com cross validation, 10 = [73.22, 79.88]


# O que vi no curso?

- Entenda os perigos do hold out <br/>
- Aplique técnicas de validação cruzada (cross validation) <br/>
- Utilize a aleatoriedade a seu favor<br/>
- Entenda quando usar diversas estratégias diferentes de validação cruzada<br/>
- KFold, StratifiedKFold, GroupKFold<br/>
- Trabalhe na previsão de novos grupos quando nem todos os dados são leituras independentes entre si<br/>
- Utilize um pipeline para treino e validação<br/>